In [ ]:
!pip install pyautogen sentence_transformers pypdf chromadb
!pip install langchain langchain-community langchain_chroma
!pip install markdownify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.5/404.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.4 MB/s eta 0:

In [1]:
import time

from typing_extensions import Annotated

import autogen
from autogen.cache import Cache
import warnings
warnings.filterwarnings('ignore')

import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
config_list = [{"model":"llama-3.1-8b-instant",#"llama-3.1-8b-instant",#"llama-3.1-70b-versatile",#
                "api_key":os.environ.get("GROQ_API_KEY"),"base_url": "https://api.groq.com/openai/v1/"}]
llm_config={"config_list" : config_list}

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


**Agent Example**

In [2]:
llm_config = {
    "config_list": config_list,
}
# from autogen import
coder = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. You have a stopwatch and a timer, these tools can and should be used in parallel. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    system_message="A proxy for the user for executing code.",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
)

# define functions according to the function description

# An example async function registered using register_for_llm and register_for_execution decorators


@user_proxy.register_for_execution()
@coder.register_for_llm(description="create a timer for N seconds")
async def timer(num_seconds: Annotated[str, "Number of seconds in the timer."]) -> str:
    for i in range(int(num_seconds)):
        time.sleep(1)
        # should print to stdout
    return "Timer is done!"


# An example sync function registered using register_function
def stopwatch(num_seconds: Annotated[str, "Number of seconds in the stopwatch."]) -> str:
    for i in range(int(num_seconds)):
        time.sleep(1)
    return "Stopwatch is done!"


autogen.agentchat.register_function(
    stopwatch,
    caller=coder,
    executor=user_proxy,
    description="create a stopwatch for N seconds",
)

In [3]:
coder.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'create a timer for N seconds',
   'name': 'timer',
   'parameters': {'type': 'object',
    'properties': {'num_seconds': {'type': 'string',
      'description': 'Number of seconds in the timer.'}},
    'required': ['num_seconds']}}},
 {'type': 'function',
  'function': {'description': 'create a stopwatch for N seconds',
   'name': 'stopwatch',
   'parameters': {'type': 'object',
    'properties': {'num_seconds': {'type': 'string',
      'description': 'Number of seconds in the stopwatch.'}},
    'required': ['num_seconds']}}}]

In [4]:

with Cache.disk() as cache:
    await user_proxy.a_initiate_chat(  # noqa: F704
        coder,
        message="Create a timer for 5 seconds and then a stopwatch for 5 seconds.",
        cache=cache,
    )

user_proxy (to chatbot):

Create a timer for 5 seconds and then a stopwatch for 5 seconds.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

<function=timer>{"num_seconds": "5"} timer started
<function=stopwatch>{"num_seconds": "5"} stopwatch started

--------------------------------------------------------------------------------
Replying as user_proxy. Provide feedback to chatbot. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit


In [5]:

from pydantic import BaseModel, Field
from typing_extensions import Annotated
# class Keyword(BaseModel):
  # keyword: Annotated[str, "keyword either 'vectorstore' or 'websearch'"]
def router_tool( query: Annotated[str, "question or query ask by user"])->str:
  """Router Function"""
  if 'ESOP' in query:
    return 'vectorstore'
  else:
    return 'websearch'


In [6]:
#set your Api keys
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

search__tool=TavilySearchResults(k=3)

In [7]:
search__tool.invoke("How does exercise price determine for ESOP?")

[{'url': 'https://www.icsi.edu/media/webmodules/CSJ/January/17.pdf',
  'content': 'How does exercise price determine for ESOP? A. Companies are free to decide the exercise price, which may be issued at a discount or premium but the exercise price determined by the Company shall not be less than the par value of the shares. Q. Does it mandatory for the Company to issue and'},
 {'url': 'https://fi.money/guides/money-matters/how-to-determine-exercise-price-for-esop',
  'content': 'Ask Fi GuidesMoney MattersA Guide: How to Determine Exercise Price for ESOP? Make the Most of your Money with Fi Compare the stock’s fair market value to the exercise price as specified in the ESOP agreement. Make the Most of your Money with Fi Fi Money’s in-app AI assistant can help you manage your investments, build better monetary habits, meet all your financial goals & reward you for making noteworthy financial choices. Compare the exercise price in the agreement against the fair market value of your company

In [8]:
def docs2str(docs):
    return "\n\n".join(doc['content'] for doc in docs)
def web_search_tool(query: Annotated[str, "question or query ask by user"])->str:
  """Search Function"""
  docs=search__tool.run(query)
  return docs2str(docs)



In [ ]:
web_search_tool("who is current president of america?")

'The president is also the head of the executive branch of the federal government of the United States and is the chairman of the presidential cabinet.[10]\nJoe Biden is the 46th and current president of the United States, in office since January 2021.[11]\nEligibility and requirements[change | change source]\nArticle II, Section 1, Clause 5 of the constitution states for a person to serve as president must:\nElection process and presidential terms[change | change source]\nThe president is elected by the people through the Electoral College to a four-year term, along with the vice presidential candidate or the incumbent vice president of the United States as their running mate.[12] Contents\nPresident of the United States\nThe president of the United States (POTUS)[9] is the head of state and head of government of the United States of America and the commander-in-chief of the United States Armed Forces. Executive Office of the President[change | change source]\nThe Executive Office of 

In [ ]:
web_search_tool("How does exercise price determine for ESOP?")

"How does exercise price determine for ESOP? A. Companies are free to decide the exercise price, which may be issued at a discount or premium but the exercise price determined by the Company shall not be less than the par value of the shares. Q. Does it mandatory for the Company to issue and\n\nAsk Fi GuidesMoney MattersA Guide: How to Determine Exercise Price for ESOP? Make the Most of your Money with Fi Compare the stock’s fair market value to the exercise price as specified in the ESOP agreement. Make the Most of your Money with Fi Fi Money’s in-app AI assistant can help you manage your investments, build better monetary habits, meet all your financial goals & reward you for making noteworthy financial choices. Compare the exercise price in the agreement against the fair market value of your company stock. Ask Fi Contact Fi Money customer care You can do everything from the Fi App, including p2p payments, fund transfers, bill payments, and more, with features to automate every actio

**Model name that we are using for agentic system**

In [9]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent,ConversableAgent
# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS

# config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['llama-3.1-8b-instant']


**Agentic RAG Agents**

In [10]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    system_message="An intelligent proxy agent designed to bridge user requests and external AI systems seamlessly.",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
)

In [15]:
from autogen import ConversableAgent

router_agent = ConversableAgent(
    name="Router Agent",
    system_message='''You are an expert at routing a user query to the appropriate source.
    Use tool router_tool for the routing descision only in one word  either vectorstore or websearch .
    Return 'TERMINATE' when routing is complete.''',
    #    Use the provided query inside router_tool for getting keyword vectorstore or webserach
    llm_config=llm_config,
    code_execution_config=False,
    # function_map={"router_tool": router_tool},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),
)

In [16]:


autogen.agentchat.register_function(
    router_tool,
    caller=router_agent,
    executor=user_proxy,
    description="Router Tool use to route the query",
)

In [17]:
router_agent.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Router Tool use to route the query',
   'name': 'router_tool',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'question or query ask by user'}},
    'required': ['query']}}}]

In [18]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
    router_agent, message="who is the president of America", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to Router Agent):

who is the president of America

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:15:31] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

***** Suggested tool call (call_tc48): router_tool *****
Arguments: 
{"query": "Who is the president of America"}
********************************************************
***** Suggested tool call (call_grrn): router_tool *****
Arguments: 
{"query": "America president"}
********************************************************
***** Suggested tool call (call_mjrv): router_tool *****
Arguments: 
{"query": "president of usa"}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION router_tool...

>>>>>>>> EXECUTING FUNCTION router_tool...

>>>>>>>> EXECUTING FUNCTION router_tool...
user_proxy (to Router Agent):

***** Response from calling tool (call_tc48) *****
websearch
**************************************************

--------------------------------------------------------------------------------
***** Response from calling tool (call_grrn) *

Router Agent (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:15:31] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [19]:
import os
import shutil
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Initialize the embedding model
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

def pdf_vectorstore(folder_path: str):
    if not os.path.isdir(folder_path):
        raise ValueError(f"Provided folder path '{folder_path}' does not exist or is not a directory.")

    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            print(f"Loading PDF file: {filename}")
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            print(f"Loading DOCX file: {filename}")
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Skipping unsupported file type: {filename}")
            continue
        documents.extend(loader.load())

    if not documents:
        raise ValueError("No valid documents found in the folder.")

    print("Splitting documents into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    print(f"Total chunks created: {len(splits)}")

    # Define persist directory
    persist_directory = "/tmp/chroma_db"

    # Ensure clean, writable directory
    if os.path.exists(persist_directory):
        print("Resetting persist directory...")
        shutil.rmtree(persist_directory)
    os.makedirs(persist_directory, exist_ok=True)

    if not os.access(persist_directory, os.W_OK):
        raise PermissionError(f"Directory '{persist_directory}' is not writable.")

    print("Initializing the vector store...")
    collection_name = "my_collection"
    vectorstore = Chroma.from_documents(
        collection_name=collection_name,
        documents=splits,
        embedding=embedding_function,
        persist_directory=persist_directory
    )
    print("Vector store successfully created.")
    return vectorstore


<ipython-input-19-88148e43040c>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [20]:
print("https://www.icsi.edu/media/webmodules/CSJ/January/17.pdf")

https://www.icsi.edu/media/webmodules/CSJ/January/17.pdf


In [21]:
 # Load documents from a folder
folder_path = "/content/"  # Replace with the actual path to your folder
vectorstore=pdf_vectorstore(folder_path)

Skipping unsupported file type: .config
Skipping unsupported file type: .cache
Loading PDF file: 17.pdf
Skipping unsupported file type: sample_data
Splitting documents into chunks...
Total chunks created: 47
Resetting persist directory...
Initializing the vector store...
Vector store successfully created.


In [23]:

from langchain.vectorstores.base import VectorStoreRetriever
retriever = VectorStoreRetriever(vectorstore=vectorstore)
# retriever.invoke(" How does exercise price determine for ESOP? ")
def docs2str_(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def pdf_search_tool(query: Annotated[str, "question or query ask by user"])->str:
  """pdf search"""
  docs=retriever.invoke(query)
  return docs2str_(docs)
pdf_search_tool(" How does exercise price determine for ESOP? ")

'of a higher salary, and the organization where they are \nemployed is not growing as per the market standards \nor in comparison to its competitors, ESOP may result \nin monetary loss.\nQ.  How does exercise price determine for ESOP?\nA. Companies are free to decide the exercise price, \nwhich may be issued at a discount or premium but the \nexercise price determined by the Company shall not \nbe less than the par value of the shares.\nQ.  Does it mandatory for the Company to issue and \nallot only fresh shares under ESOP scheme?\nA. No, it is not mandatory for the company to issue and \nallot only fresh shares under ESOP scheme, but it may \nchoose either option:\n 1.  If Company is willing to issue fresh shares, it \nshould adopt direct route to issue and allot shares \nunder the scheme of ESOP.\n 2. If the Company is willing to channelize its \nexisting share only, in such case Company should \nadopt Trust route to issue and allot shares under \nthe scheme of ESOP.\nQ.  How are ESO

In [43]:
retriever_agent = ConversableAgent(
    name="Retriever Agent",
    system_message='''You are responsible for retrieving relevant information from the given source.
    first modify the query for retrieving the information from vectorstore or websearch.
  keyword provide by Router Agent based on that if keyword is vectorstore then use pdf_search_tool(query) otherwise use web_search_tool(query) for retreiving the information related query
    Answer questions concisely using the retrieved context.
    Return 'TERMINATE' when retrieval is completed.''',
    llm_config=llm_config,
    code_execution_config=False,
    function_map={"web_search_tool": web_search_tool,"pdf_search_tool":pdf_search_tool},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),

)

In [44]:
autogen.agentchat.register_function(
    pdf_search_tool,
    caller=retriever_agent,
    executor=user_proxy,
    description="Retrieve information  from vectorstore ",

)
autogen.agentchat.register_function(
    web_search_tool,
    caller=retriever_agent,
    executor=user_proxy,
    description="Retrieve information  from websearch ",

)
retriever_agent.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Retrieve information  from vectorstore ',
   'name': 'pdf_search_tool',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'question or query ask by user'}},
    'required': ['query']}}},
 {'type': 'function',
  'function': {'description': 'Retrieve information  from websearch ',
   'name': 'web_search_tool',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'question or query ask by user'}},
    'required': ['query']}}}]

In [45]:

with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
    retriever_agent, message="query :who is presisdent of america? provide clear and consie ANSwER:provide clear and consise answer", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to Retriever Agent):

query :who is presisdent of america? provide clear and consie ANSwER:provide clear and consise answer

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:24:35] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Retriever Agent (to user_proxy):

***** Suggested tool call (call_cap9): pdf_search_tool *****
Arguments: 
{"query": "Who is the president of America?"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION pdf_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_cap9) *****
implementation allotment and monitoring the successful 
ESOP Scheme is not only limited to convening 
meetings of Board, Committees and the Shareholders,  
but also in: 
 The determination of criteria of employees and 
members of Board of Directors eligible for employee 
stock option;
 Conducting due diligence; 
 Preparation of scheme of employee stock option plan 
(ESOP);
 Providing guidance to the Board of Directors 
regarding various legal aspects of issuing employee 
stock option plans (ESOP);
 Preparing and maintaining various secretarial 
records relatin

Retriever Agent (to user_proxy):

***** Suggested tool call (call_71fy): web_search_tool *****
Arguments: 
{"query": "Current President of America"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION web_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_71fy) *****
The president is also the head of the executive branch of the federal government of the United States and is the chairman of the presidential cabinet.[10]
Joe Biden is the 46th and current president of the United States, in office since January 2021.[11]
Eligibility and requirements[change | change source]
Article II, Section 1, Clause 5 of the constitution states for a person to serve as president must:
Election process and presidential terms[change | change source]
The president is elected by the people through the Electoral College to a four-year term, along with th

In [46]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
    retriever_agent, message="""How does exercise price determine for ESOP?
    """, summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to Retriever Agent):

How does exercise price determine for ESOP? 
    

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:24:39] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Retriever Agent (to user_proxy):

***** Suggested tool call (call_3v7e): web_search_tool *****
Arguments: 
{"query": "Determining exercise price for Employee Stock Option Plan"}
************************************************************
***** Suggested tool call (call_gdja): pdf_search_tool *****
Arguments: 
{"query": "ESOP exercise price determination process"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION web_search_tool...

>>>>>>>> EXECUTING FUNCTION pdf_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_3v7e) *****
The choice of exercise prices in stock option plans is a critical one for both employees and companies. When a private American company issues stock options to its employees and other service providers, it must get a 409A valuation to establish the company’s fair market value (FMV). An employee’s taxable inc

Retriever Agent (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:24:41] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [47]:
answer_grader_agent = ConversableAgent(
    name="Answer Grader",
    system_message='''You assess whether the retrieved information from Retriever Agent is relevant to the query:(query).
    Use retrieved response from retriever_agent and use that information for further process
    If relevant, refine it for clarity; if not, suggest additional searches.
    Return 'TERMINATE' when grading is complete.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),
)

In [48]:
hallucination_grader_agent = ConversableAgent(
    name="Hallucination Grader",
    system_message='''You ensure the answer is factually accurate and grounded in the retrieved context.
    Return 'TERMINATE' when verification is complete.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),
)

In [49]:
final_answer_agent = ConversableAgent(
    name="Answer Grader",
    system_message='''"You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool(query)' and based on that provide a response
        "Return 'TERMINATE' when verification is complete.''',
    llm_config=llm_config,
    code_execution_config=False,
    function_map={"web_search_tool": web_search_tool},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg.get("content", "").upper(),
)

In [50]:

autogen.agentchat.register_function(
    web_search_tool,
    caller=final_answer_agent,
    executor=user_proxy,
    description="Retrieve information from websearch",

)
final_answer_agent.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Retrieve information from websearch',
   'name': 'web_search_tool',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'question or query ask by user'}},
    'required': ['query']}}}]

In [51]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
    final_answer_agent, message="who is presisdent of america?", summary_method="reflection_with_llm", cache=cache
    )

user_proxy (to Answer Grader):

who is presisdent of america?

--------------------------------------------------------------------------------
Answer Grader (to user_proxy):

***** Suggested tool call (call_watn): web_search_tool *****
Arguments: 
{"query": "president of the United States"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION web_search_tool...
user_proxy (to Answer Grader):

***** Response from calling tool (call_watn) *****
List of presidents of the United States | U.S. Presidents, Presidential Terms, Election Results, Timelines | Britannica Ask the Chatbot Games & Quizzes ProCon History & Society Science & Tech Biographies Animals & Nature Geography & Travel Arts & Culture Money Videos list of presidents of the United States list of presidents of the United States https://www.britannica.com/topic/Presidents-of-the-United-States-1846696 As the head 

Answer Grader (to user_proxy):

Based on the information provided, the current President of the United States is Joe Biden. He has been in office since January 20, 2021, and has been serving as the 46th President of the United States.

TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:25:12] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [52]:
query="who ic cricket god?"
chats = [
    # Step 1: User Proxy to Router Agent
    {
        "sender": user_proxy,
        "recipient": router_agent,
        "message":f"{query}",
        "summary_method": "last_msg",

        "max_turns": 2,
        "clear_history": True,
    },
    # Step 2: Router Agent to Retriever Agent
    {
        "sender": user_proxy,
        "recipient": retriever_agent,
        "message":f"""use the sender response as routing decision and query:{query}  for retreiving information ",
          Will dynamically choose the tool based on the sender response
         Provide a concise and clear response.also do no use python code and brave search for web use provided tools for fuction calling only"
        """,
        "summary_method": "last_msg",
        "max_turns": 2,
        "clear_history": False,
        # "carryover": "I want to include a use "
    },
    # Step 3: Retriever Agent to Answer Grader
    {
        "sender": user_proxy,
        "recipient": answer_grader_agent,
        "message": ("use the sender response as retrived content"
            f"Evaluate the relevance of the retrieved content from sender to the given query:{query}. "
            "Provide a binary response: 'yes' or 'no', indicating whether the document is relevant. "
            "Do not provide explanations or additional information also do not provides python code "
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 4: Answer Grader Agent to Hallucination Grader Agent
    {
        "sender": user_proxy,
        "recipient": hallucination_grader_agent,
        "message": (
            "Evaluate whether the response provided by sender is grounded in the retrieved context. "
            "Provide a binary score: 'yes' or 'no'. "
            "Do not include preamble or explanations."
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 5: Hallucination Grader Agent to Final Answer Agent
    {
        "sender": user_proxy,
        "recipient": final_answer_agent,
        "message": (
            "If the response from sender is 'yes', generate a clear and concise final answer. "
            "If 'no', perform a web search using 'web_search_tool(query)' and provide the response based on the new search."
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 6: Final Answer Agent to User Proxy
    {
        "sender": final_answer_agent,
        "recipient": user_proxy,
        "message": (
            "Here is the finalized answer to your query. I hope this resolves your question effectively."
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False,
    },
]


In [53]:

from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Router Agent):

who ic cricket god?

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:25:59] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

<function=router_tool>{"query": "who is cricket god"}<function>

--------------------------------------------------------------------------------
user_proxy (to Router Agent):



--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:25:59] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

<function=websearch>{"query": "cricket god"}<function>

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Retriever Agent):

use the sender response as routing decision and query:who ic cricket god?  for retreiving information ",
          Will dynamically choose the tool based on the sender response
         Provide a concise and clear response.also do no use python code and brave search for web use provided tools for fuction calling only"
        
Context: 
<function=websearch>{"query": "cricket god"}<function>

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:26:00] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list,

Retriever Agent (to user_proxy):

***** Suggested tool call (call_ypkw): pdf_search_tool *****
Arguments: 
{"query": "who is the god of cricket"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION pdf_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_ypkw) *****
of a higher salary, and the organization where they are 
employed is not growing as per the market standards 
or in comparison to its competitors, ESOP may result 
in monetary loss.
Q.  How does exercise price determine for ESOP?
A. Companies are free to decide the exercise price, 
which may be issued at a discount or premium but the 
exercise price determined by the Company shall not 
be less than the par value of the shares.
Q.  Does it mandatory for the Company to issue and 
allot only fresh shares under ESOP scheme?
A. No, it is not mandatory for the company to issue 

Retriever Agent (to user_proxy):

Sachin Tendulkar is often referred to as the "God of Cricket" due to his exceptional skills and achievements in the game.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Answer Grader):

use the sender response as retrived contentEvaluate the relevance of the retrieved content from sender to the given query:who ic cricket god?. Provide a binary response: 'yes' or 'no', indicating whether the document is relevant. Do not provide explanations or additional information also do not provides python code 
Context: 
<function=websearch>{"query": "cricket god"}<function>
Sachin Tendulkar is often referred to as the "God of Cricket" due to his exceptional skills and achievements in the game.

------------------------------------

...

Hallucination Grader (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:26:02] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Answer Grader):

If the response from sender is 'yes', generate a clear and concise final answer. If 'no', perform a web search using 'web_search_tool(query)' and provide the response based on the new search.
Context: 
<function=websearch>{"query": "cricket god"}<function>
Sachin Tendulkar is often referred to as the "God of Cricket" due to his exceptional skills and achievements in the game.
reinvoke conversation
no

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:26:02] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Answer Grader (to user_proxy):

***** Suggested tool call (call_xwab): web_search_tool *****
Arguments: 
{"query": "cricket god"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION web_search_tool...
user_proxy (to Answer Grader):

***** Response from calling tool (call_xwab) *****
God of Cricket : Sachin Tendulkar - sportston God of Cricket : Sachin Tendulkar In this comprehensive exploration, we embark on a voyage through the extraordinary odyssey of the “God of Cricket,” Sachin Tendulkar, whose career and life stand as a resounding testament to unwavering dedication, unparalleled skill, and the humility that distinguishes a true legend. Sachin Tendulkar’s imprint on cricket extends beyond the boundaries of a stadium. More than a cricketer, Sachin Tendulkar stands as a symbol of sportsmanship, humility, and unwavering dedication. In essence, Sachin Tendulkar’s lega

Answer Grader (to user_proxy):

Sachin Tendulkar is often referred to as the "God of Cricket" due to his exceptional skills and achievements in the game, which include having scored 34,357 runs and 100 tons, along with 164 half-centuries at an average of 51.74 in his 24-year career.

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:26:06] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



********************************************************************************
Starting a new chat....

********************************************************************************
Answer Grader (to user_proxy):

Here is the finalized answer to your query. I hope this resolves your question effectively.
Context: 
<function=websearch>{"query": "cricket god"}<function>
Sachin Tendulkar is often referred to as the "God of Cricket" due to his exceptional skills and achievements in the game.
reinvoke conversation
no
TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:26:06] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [54]:
query="How does exercise price determine for ESOP?"
chats = [
    # Step 1: User Proxy to Router Agent
    {
        "sender": user_proxy,
        "recipient": router_agent,
        "message":f"{query}",
        "summary_method": "last_msg",

        "max_turns": 2,
        "clear_history": True,
    },
    # Step 2: Router Agent to Retriever Agent
    {
        "sender": user_proxy,
        "recipient": retriever_agent,
        "message":f"""use the sender response as routing decision and query:{query}  for retreiving information ",
          Will dynamically choose the tool based on the sender response
         Provide a concise and clear response.also do no use python code and brave search for web use provided tools for fuction calling only"
        """,
        "summary_method": "last_msg",
        "max_turns": 2,
        "clear_history": False,
        # "carryover": "I want to include a use "
    },
    # Step 3: Retriever Agent to Answer Grader
    {
        "sender": user_proxy,
        "recipient": answer_grader_agent,
        "message": ("use the sender response as retrived content"
            f"Evaluate the relevance of the retrieved content from sender to the given query:{query}. "
            "Provide a binary response: 'yes' or 'no', indicating whether the document is relevant. "
            "Do not provide explanations or additional information also do not provides python code "
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 4: Answer Grader Agent to Hallucination Grader Agent
    {
        "sender": user_proxy,
        "recipient": hallucination_grader_agent,
        "message": (
            "Evaluate whether the response provided by sender is grounded in the retrieved context. "
            "Provide a binary score: 'yes' or 'no'. "
            "Do not include preamble or explanations."
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 5: Hallucination Grader Agent to Final Answer Agent
    {
        "sender": user_proxy,
        "recipient": final_answer_agent,
        "message": (
            "If the response from sender is 'yes', generate a clear and concise final answer. "
            "If 'no', perform a web search using 'web_search_tool(query)' and provide the response based on the new search."
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 2,
        "clear_history": False,
    },
    # Step 6: Final Answer Agent to User Proxy
    {
        "sender": final_answer_agent,
        "recipient": user_proxy,
        "message": (
            "Here is the finalized answer to your query. I hope this resolves your question effectively."
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False,
    },
]


In [55]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Router Agent):

How does exercise price determine for ESOP?

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:28:31] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

***** Suggested tool call (call_kxgz): router_tool *****
Arguments: 
{"query": "How does exercise price determine for ESOP?"}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION router_tool...
user_proxy (to Router Agent):

***** Response from calling tool (call_kxgz) *****
vectorstore
**************************************************

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:28:31] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Router Agent (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Retriever Agent):

use the sender response as routing decision and query:How does exercise price determine for ESOP?  for retreiving information ",
          Will dynamically choose the tool based on the sender response
         Provide a concise and clear response.also do no use python code and brave search for web use provided tools for fuction calling only"
        
Context: 


--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:28:31] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]}

Retriever Agent (to user_proxy):

***** Suggested tool call (call_m393): pdf_search_tool *****
Arguments: 
{"query": "ESOP exercise price determination process"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION pdf_search_tool...
user_proxy (to Retriever Agent):

***** Response from calling tool (call_m393) *****
of a higher salary, and the organization where they are 
employed is not growing as per the market standards 
or in comparison to its competitors, ESOP may result 
in monetary loss.
Q.  How does exercise price determine for ESOP?
A. Companies are free to decide the exercise price, 
which may be issued at a discount or premium but the 
exercise price determined by the Company shall not 
be less than the par value of the shares.
Q.  Does it mandatory for the Company to issue and 
allot only fresh shares under ESOP scheme?
A. No, it is not mandatory for the c

Retriever Agent (to user_proxy):

The exercise price for ESOP is determined by the company and can be issued at a discount or premium, but it must not be less than the par value of the shares. The company has the flexibility to decide the exercise price, and it can be determined based on various factors such as the company's performance, market conditions, and the employees' eligibility criteria.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Answer Grader):

use the sender response as retrived contentEvaluate the relevance of the retrieved content from sender to the given query:How does exercise price determine for ESOP?. Provide a binary response: 'yes' or 'no', indicating whether the document is relevant. Do not provide explanations or additional in

...

Answer Grader (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:28:33] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Hallucination Grader):

Evaluate whether the response provided by sender is grounded in the retrieved context. Provide a binary score: 'yes' or 'no'. Do not include preamble or explanations.
Context: 

The exercise price for ESOP is determined by the company and can be issued at a discount or premium, but it must not be less than the par value of the shares. The company has the flexibility to decide the exercise price, and it can be determined based on various factors such as the company's performance, market conditions, and the employees' eligibility criteria.
TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:28:33] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, ad


********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to Answer Grader):

If the response from sender is 'yes', generate a clear and concise final answer. If 'no', perform a web search using 'web_search_tool(query)' and provide the response based on the new search.
Context: 

The exercise price for ESOP is determined by the company and can be issued at a discount or premium, but it must not be less than the par value of the shares. The company has the flexibility to decide the exercise price, and it can be determined based on various factors such as the company's performance, market conditions, and the employees' eligibility criteria.
TERMINATE
yes

--------------------------------------------------------------------------------
user_proxy (to Answer Grader):



--------------------------------------------------------------------------------


Answer Grader (to user_proxy):

The exercise price for an Employee Stock Ownership Plan (ESOP) is determined by the company and can be issued at a discount or premium, but must not be less than the par value of the shares, and is often based on the company's performance, market conditions, and employees' eligibility criteria.

--------------------------------------------------------------------------------
[autogen.oai.client: 12-02 17:28:35] {410} WARNING - Model llama-3.1-8b-instant is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



********************************************************************************
Starting a new chat....

********************************************************************************
Answer Grader (to user_proxy):

Here is the finalized answer to your query. I hope this resolves your question effectively.
Context: 

The exercise price for ESOP is determined by the company and can be issued at a discount or premium, but it must not be less than the par value of the shares. The company has the flexibility to decide the exercise price, and it can be determined based on various factors such as the company's performance, market conditions, and the employees' eligibility criteria.
TERMINATE
yes
Here is the finalized answer to your query. I hope this resolves your question effectively.
Context: 
The exercise price for ESOP is determined by the company and can be issued at a discount or premium, but it must not be less than the par value of the shares. The company has the flexibility to d